In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import geopandas
import ee
import geemap
from utils import clipAndResample
#ee.Initialize()
import xarray as xr
import rioxarray

In [13]:
# find new Landcover (NOAA) values at sample points
file = pd.read_csv('poc.csv')
coords = np.array(file[['Long', 'Lat']])

lc_noaa_src = rasterio.open('Low_res_data_05/landcover_noaa_afr_05.tif')
lc_noaa_samples = [item for sublist in list(lc_noaa_src.sample(coords)) for item in sublist]

In [ ]:
# code for creating a mean of tiffs

from osgeo import gdal
import numpy as np

def write_raster(raster_array, gt, data_obj, outputpath, dtype=gdal.GDT_UInt16, options=0, color_table=0, nbands=1, nodata=False):

    height, width = raster_array.shape

    # Prepare destination file
    driver = gdal.GetDriverByName("GTiff")
    if options != 0:
        dest = driver.Create(outputpath, width, height, nbands, dtype, options)
    else:
        dest = driver.Create(outputpath, width, height, nbands, dtype)

    # Write output raster
    if color_table != 0:
        dest.GetRasterBand(1).SetColorTable(color_table)

    dest.GetRasterBand(1).WriteArray(raster_array)

    if nodata is not False:
        dest.GetRasterBand(1).SetNoDataValue(nodata)

    # Set transform and projection
    dest.SetGeoTransform(gt)
    wkt = data_obj.GetProjection()
    srs = osr.SpatialReference()
    srs.ImportFromWkt(wkt)
    dest.SetProjection(srs.ExportToWkt())

    # Close output raster dataset 
    dest = None

tifflist = ['something1.tif', 'something2.tif'] # the paths to each of the tiffs you want to use in averaging
for i, tiff in enumerate(tifflist):
    gd_obj = gdal.Open(tiff)
    array = gd_obj.ReadAsArray()
    array = np.expand_dims(array,2)
    if i == 0:
        allarrays = array
    else:
        allarrays = np.concatenate((allarrays, array), axis=2)
mean_of_tiffs = np.nansum(allarrays, axis=2)

outputpath = 'wherever you are saving this guy.tif'
write_raster(mean_of_tiffs, gd_obj.GetGeoTransform(), gd_obj, outputpath)

In [17]:
# find annual average of GPCC precip dataset
from netCDF4 import Dataset
gpcc_dataset = xr.open_dataset("GPCC/out.nc") # lon(720), lat(360), time(360)
yearly_prec_nc = gpcc_dataset.resample(time='1y').sum()
mean_prec_nc = yearly_prec_nc.mean(dim='time')
mean_prec_nc['precip'].rio.to_raster('LTA_GPCC_precip.tif')

In [59]:
mean_prec_nc['precip'].rio.to_raster('LTA_GPCC_precip.tif')

In [58]:
mean_prec_nc.get('precip').isel(lon=379, lat=169)

<xarray.DataArray 'precip' ()>
array(2294.9387, dtype=float32)
Coordinates:
    lon      float64 9.75
    lat      float64 5.25

In [172]:
# find annual average of GPCP dataset
gpcp_dataset = xr.open_dataset("GPCP/out.nc") # lon(720), lat(360), time(360)
days = [31,28,31,30,31,30,31,31,30,31,30,31, 31,28,31,30,31,30,31,31,30,31,30,31, 31,28,31,30,31,30,31,31,30,31,30,31, 31,29,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,29,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,29,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,29,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,29,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,29,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,29,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31]
for (i,d) in enumerate(days):
    gpcp_dataset['precip'][i] = gpcp_dataset['precip'][i] * d
yearly_prec_nc = gpcp_dataset.resample(time='1y').sum()
mean_prec_nc = yearly_prec_nc.resample(time='29y').mean()

mean_prec_nc['precip'].rio.to_raster('LTA_GPCP_precip_25.tif')

In [171]:
mean_prec_nc['precip']

<xarray.DataArray 'precip' (time: 2, latitude: 72, longitude: 144)>
array([[[129.19537 , 128.15681 , 126.38945 , ..., 133.26077 ,
         141.00227 , 143.09848 ],
        [ 84.29381 ,  89.40729 ,  88.67485 , ...,  84.632454,
          92.81697 ,  87.94917 ],
        [ 56.81002 ,  61.727043,  57.33716 , ...,  64.34771 ,
          71.476944,  66.36371 ],
        ...,
        [199.64218 , 200.10161 , 199.81633 , ..., 190.7377  ,
         186.74507 , 189.91205 ],
        [176.13716 , 180.60878 , 179.31021 , ..., 172.73724 ,
         176.88115 , 176.93964 ],
        [177.71764 , 179.33127 , 174.82571 , ..., 171.5885  ,
         176.74818 , 175.42719 ]],

       [[109.505585, 106.912415, 105.28148 , ..., 109.71927 ,
         116.9897  , 116.55097 ],
        [ 77.33448 ,  78.16896 ,  74.30289 , ...,  76.49201 ,
          80.23284 ,  78.70794 ],
        [ 65.09852 ,  64.2796  ,  55.134758, ...,  72.02436 ,
          72.59897 ,  70.23787 ],
        ...,
        [181.72163 , 181.6312  , 183.65642 , ..., 180.61089 ,
         179.01813 , 178.71059 ],
        [166.9692  , 169.3949  , 169.47812 , ..., 166.72153 ,
         168.54349 , 168.74545 ],
        [177.38702 , 179.81345 , 177.86757 , ..., 173.65312 ,
         175.26546 , 176.39352 ]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 1981-12-31 2010-12-31
  * latitude   (latitude) float32 -88.75 -86.25 -83.75 ... 83.75 86.25 88.75
  * longitude  (longitude) float32 1.25 3.75 6.25 8.75 ... 353.8 356.2 358.8

In [143]:
help(gpcp_dataset.resample)

Help on method resample in module xarray.core.common:

resample(indexer: Mapping[Hashable, str] = None, skipna=None, closed: str = None, label: str = None, base: int = 0, keep_attrs: bool = None, loffset=None, restore_coord_dims: bool = None, **indexer_kwargs: str) method of xarray.core.dataset.Dataset instance
    Returns a Resample object for performing resampling operations.
    
    Handles both downsampling and upsampling. The resampled
    dimension must be a datetime-like coordinate. If any intervals
    contain no values from the original object, they will be given
    the value ``NaN``.
    
    Parameters
    ----------
    indexer : {dim: freq}, optional
        Mapping from the dimension name to resample frequency [1]_. The
        dimension must be datetime-like.
    skipna : bool, optional
        Whether to skip missing values when aggregating in downsampling.
    closed : {"left", "right"}, optional
        Side of each interval to treat as closed.
    label : {"left", 

In [10]:
clipAndResample('Recharge_files/LTA_soil_moisture_Africa.tif', 'CRU_precip_CGIAR_AI_data/Africa_continent_shape.shp', 
                'SM.tif', 1/2)

In [24]:
clipAndResample('Recharge_files/LTA_NDVI.asc', 'CRU_precip_CGIAR_AI_data/Africa_continent_shape.shp', 
                'test.tif', 1/10)

In [8]:
with rasterio.open('Low_res_data_05/LTA_FLDAS_Soil_moisture_Afr_05.tif') as src:
    print(np.count_nonzero(~np.isnan(src.read())))

24336


In [39]:
with rasterio.open('Recharge_files/GLOBCOVER_L4_200901_200912_V2.3_AfricaClip.tif') as src:
    print(src.read().shape)

(1, 26424, 32352)


In [2]:
with rasterio.open('High_res_data_01/RF_recharge.tif') as src:
    print(src.read().shape)

RasterioIOError: High_res_data_01/RF_recharge.tif: No such file or directory

In [25]:
# reproject
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

dst_crs = 'EPSG:4326'

with rasterio.open('CPC_ARC/africa_arc.19840104.tif') as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open('CPC_ARC/africa_arc.19840104_n.tif', 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

In [67]:
with rasterio.open('Precip_sensitivity/LTA_GPCC_precip_05.tif') as src:
    print(src.crs)
    print(src.read().shape)

EPSG:4326
(2, 145, 178)


In [10]:
# convert .asc to .tif
import rasterio.crs
import rasterio.mask
from rasterio.enums import Resampling
import fiona
from rasterio.io import MemoryFile

# read continent outline
with fiona.open("CRU_precip_CGIAR_AI_data/Africa_continent_shape.shp", "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

crs = rasterio.crs.CRS({"init": "epsg:4326"})

# define the scaling factor
scale_factor = 1/5

# read text file
with rasterio.open('High_res_data_01/landcover_Afr_01.tif', mode='r+') as src:
    #src.crs = crs
    # mask out the ocean
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True, all_touched=True)
    out_meta = src.meta
    # config for intermediate in-memory dataset
    out_meta.update({"height": out_image.shape[1], "width": out_image.shape[2],"transform": out_transform})
    
    # create in-memory cropped raster
    with MemoryFile() as memfile:
        with memfile.open(**out_meta) as dataset:  # Open as DatasetWriter
            dataset.write(out_image)
        with memfile.open() as dataset:  # Reopen as DatasetReader
            # resample data to target shape
            data = dataset.read(
                out_shape=(
                    dataset.count,
                    int(dataset.height * scale_factor),
                    int(dataset.width * scale_factor)
                ),
                resampling=Resampling.mode
            )

            # scale image transform
            transform = dataset.transform * dataset.transform.scale(
                (dataset.width / data.shape[-1]),
                (dataset.height / data.shape[-2])
            )
            # final raster config
            profile = dataset.profile
            profile.update(transform=transform, driver='GTiff', 
                height=int(dataset.height * scale_factor), width=int(dataset.width * scale_factor))

# write file
with rasterio.open("Low_res_data_05/landcover_Afr_05_2.tif", "w", **profile) as dest:
    dest.write(data)

In [5]:
# clip
import rasterio.crs
import rasterio.mask
from rasterio.enums import Resampling
import fiona
from rasterio.io import MemoryFile

# read continent outline
with fiona.open("CRU_precip_CGIAR_AI_data/Africa_continent_shape.shp", "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

crs = rasterio.crs.CRS({"init": "epsg:4326"})

# read text file
with rasterio.open('LTA_GPCC_precip.tif', mode='r+') as src:
    src.crs = crs
    # mask out the ocean
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True, all_touched=True)
    out_meta = src.meta
    # config for intermediate in-memory dataset
    out_meta.update({"height": out_image.shape[1], "width": out_image.shape[2],"transform": out_transform})

# write file
with rasterio.open("Precip_sensitivity/LTA_GPCC_precip_Africa_05.tif", "w", **out_meta) as dest:
    dest.write(out_image)

In [41]:
# convert .asc to .tif
import rasterio.crs
import rasterio.mask
from rasterio.enums import Resampling
import fiona
from rasterio.io import MemoryFile

# define the scaling factor
scale_factor = 1/150

# read text file
with rasterio.open('Recharge_files/GLOBCOVER_L4_200901_200912_V2.3_AfricaClip.tif', mode='r') as dataset:
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * scale_factor),
            int(dataset.width * scale_factor)
        ),
        resampling=Resampling.mode
    )
    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )
    # final raster config
    profile = dataset.profile
    profile.update(transform=transform, driver='GTiff', 
                   height=int(dataset.height * scale_factor), width=int(dataset.width * scale_factor))

# write file
with rasterio.open("Low_res_data_05/landcover_Afr_05.tif", "w", **profile) as dest:
    dest.write(data)

In [23]:
# # get and resample digital elevation data
# ee_dem = ee.Image("CGIAR/SRTM90_V4").rename('dem')

# # Africa only
# geometry = ee.Geometry.Polygon([[[-29.262235, -37.679961],
#     [68.400326, -37.679961],
#     [68.400326, 38.738848],
#     [-29.262235, 38.738848],
#     [-29.262235, -37.679961]]], evenOdd=False)
# featureCollection = geemap.vector_to_ee('CRU_precip_CGIAR_AI_data/Africa_continent_shape.shp')
# ee_dem_africa = ee_dem.clip(geometry)
# ee_dem_ll = ee_dem.resample('bilinear').reproject(crs='EPSG:4326', scale=50000.0)

# print(ee_dem_africa)
# #geemap.ee_export_image(ee_dem_ll, 'SRTM5000.tif')

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.clip",
    "arguments": {
      "geometry": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Polygon",
          "arguments": {
            "coordinates": {
              "constantValue": [
                [
                  [
                    -29.262235,
                    -37.679961
                  ],
                  [
                    68.400326,
                    -37.679961
                  ],
                  [
                    68.400326,
                    38.738848
                  ],
                  [
                    -29.262235,
                    38.738848
                  ],
                  [
                    -29.262235,
                    -37.679961
                  ]
                ]
              ]
            },
            "crs": {
              "functionInvocationValue": {
                "functionName": "Projection",
           